<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4b66dc9384b934e58c08736b817c880147802e97f870b0af286454aab409c101
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 15:42:19
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -11.88 K (-0.08%)
Current PnL: -19.63 L (-13.08%)
CY Booked + Current PnL: -7.53 L (-5.02%)
-------------------
Total profit:  2.24 L
Total loss:  -21.87 L
-------------------
Total Booked + Current PnL: 19.45 L (15.8%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.52%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.81 L (61.12%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-9.53,66.0,X-LC,3.18,226450.0,16084.0,12704.0,0.22,7.65,5.61,13.68,37.0,1.27,1.60,25.43,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,3.12,60.0,X-MC,2.80,216210.0,24468.0,13297.0,0.71,12.76,6.15,19.69,99.0,1.84,1.53,20.43,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-9.04,44.0,X-MC,3.32,89790.0,-888.0,15794.0,-0.61,-0.98,17.59,16.44,101.0,-0.06,0.63,19.00,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,96.16,49.0,M-SC,1.97,84482.0,-16295.0,16390.0,0.48,-16.17,19.40,0.09,245.0,-0.99,0.60,10.99,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,64.0,X-LC,1.95,203815.0,3677.0,17222.0,0.20,1.84,8.45,10.44,4.0,0.21,1.44,7.50,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-197.73,81.0,M-SC,24.62,169425.0,20205.0,123968.0,7.94,13.54,73.17,96.62,208.0,0.16,1.20,83.35,XY25,NTT,FINANCE
70,SURYODAY,240.00,71.05,65.0,H-SC,12.29,156794.0,-22277.0,88008.0,3.30,-12.44,56.13,36.71,135.0,-0.25,1.11,55.57,XR,NTT,BANKS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,27.02,82925.0,-30624.0,70677.0,2.64,-26.97,85.23,35.27,268.0,-0.43,0.59,102.96,XR,NTT,HOTELS
82,VAIBHAVGBL,521.00,55.37,61.0,H-SC,4.40,134644.0,-48131.0,159203.0,1.93,-26.33,118.24,60.77,125.0,-0.30,0.95,20.60,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,9.40,71.0,H-SC,5.24,130460.0,5163.0,53763.0,1.79,4.12,41.21,47.03,141.0,0.10,0.92,63.35,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,224.94,51.0,M-SC,5.29,145156.0,-37922.0,116894.0,-3.30,-20.71,80.53,43.14,230.0,-0.32,1.03,36.54,XY24,NTT,MISC
31,HINDUNILVR,2922.00,-10.34,44.0,X-LC,7.67,251640.0,-5423.0,40564.0,-3.27,-2.11,16.12,13.67,24.0,-0.13,1.78,17.87,XY25,NTT,FMCG
7,ASIANTILES,137.00,7050.00,62.0,L-SC,7.18,78459.0,-15351.0,91970.0,-2.70,-16.36,117.22,81.67,269.0,-0.17,0.55,51.94,XR,NTT,CERAMICS
21,COLPAL,3767.14,-4.10,46.0,X-MC,8.14,223940.0,-39425.0,152772.0,-2.08,-14.97,68.22,43.04,84.0,-0.26,1.58,3.96,XY25,ATH,FMCG
86,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,3.86,102795.0,11296.0,67454.0,-2.06,12.35,65.62,86.07,223.0,0.17,0.73,49.28,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,41.14,46.0,M-MC,9.99,222930.0,-2032.0,169694.0,0.09,-0.90,76.12,74.53,192.0,-0.01,1.58,31.06,XY24,BTT,STEEL
36,IEX,219.0,-30.16,62.0,H-SC,17.42,199547.0,-407.0,97638.0,1.75,-0.20,48.93,48.63,115.0,-0.00,1.41,11.33,XR,NTT,MISC
56,RECLTD,446.0,45.36,46.0,M-LC,5.70,204930.0,1760.0,40371.0,-0.61,0.87,19.70,20.74,55.0,0.04,1.45,6.46,XY25,NTT,FINANCE
87,WIPRO,420.0,-14.25,46.0,M-LC,5.94,150648.0,-297.0,109747.0,-0.54,-0.20,72.85,72.51,53.0,-0.00,1.06,5.81,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,122.87,69.0,H-SC,15.26,140454.0,-2025.0,21546.0,0.23,-1.42,15.34,13.70,163.0,-0.09,0.99,65.77,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,59.0,H-SC,2.38,228573.0,-41094.0,77418.0,-0.57,-15.24,33.87,13.47,138.0,-0.53,1.62,16.22,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,96.16,49.0,M-SC,1.97,84482.0,-16295.0,16390.0,0.48,-16.17,19.40,0.09,245.0,-0.99,0.60,10.99,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-22.20,55.0,H-MC,7.48,105336.0,-26499.0,68721.0,1.33,-20.10,65.24,32.03,98.0,-0.39,0.74,17.99,OX40N,NTT,IT
18,CERA,9475.0,-23.39,45.0,H-SC,2.62,139656.0,-36247.0,78263.0,0.83,-20.61,56.04,23.89,149.0,-0.46,0.99,20.61,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,46.0,M-LC,5.70,204930.0,1760.0,40371.0,-0.61,0.87,19.70,20.74,55.0,0.04,1.45,6.46,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.80,53.0,M-LC,11.00,211714.0,6638.0,105921.0,0.94,3.24,50.03,54.89,52.0,0.06,1.50,28.11,X5K,ATH,METALS
46,KPIGREEN,731.05,9.40,71.0,H-SC,5.24,130460.0,5163.0,53763.0,1.79,4.12,41.21,47.03,141.0,0.10,0.92,63.35,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,58.0,M-LC,9.09,161192.0,20352.0,24130.0,-0.76,14.45,14.97,31.59,66.0,0.84,1.14,44.77,XY24,NTT,BANKS
12,BANKINDIA,190.00,-20.50,68.0,H-MC,14.25,205938.0,26130.0,86288.0,-1.20,14.53,41.90,62.52,88.0,0.30,1.46,53.57,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-20.50,68.0,H-MC,14.25,205938.0,26130.0,86288.0,-1.20,14.53,41.90,62.52,88.0,0.30,1.46,53.57,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,3.86,102795.0,11296.0,67454.0,-2.06,12.35,65.62,86.07,223.0,0.17,0.73,49.28,XR,NTT,DURABLES
46,KPIGREEN,731.05,9.40,71.0,H-SC,5.24,130460.0,5163.0,53763.0,1.79,4.12,41.21,47.03,141.0,0.10,0.92,63.35,MH,ATH,POWER
32,HINDZINC,730.22,29.80,53.0,M-LC,11.00,211714.0,6638.0,105921.0,0.94,3.24,50.03,54.89,52.0,0.06,1.50,28.11,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.01,45.0,H-SC,7.17,125652.0,2316.0,129308.0,0.89,1.88,102.91,106.72,119.0,0.02,0.89,25.70,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147324.0,-129199.0,239269.0,1.00,-46.72,162.41,39.81,54.0,-0.54,1.04,3.84,XY24,NTT,AUTO
24,DMART,5391.45,-17.31,33.0,X-LC,10.16,101134.0,-1452.0,28257.0,-1.27,-1.42,27.94,26.13,38.0,-0.05,0.71,23.79,X40N,ATH,FMCG
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51268.0,-13738.0,164668.0,-0.38,-21.13,321.19,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.42,40.0,X-MC,6.79,82050.0,-1460.0,21489.0,-1.05,-1.75,26.19,23.98,82.0,-0.07,0.58,3.97,X40,ATH,APPARELS
52,PGHH,17907.65,-29.75,41.0,X-MC,3.21,203280.0,2460.0,65334.0,-1.12,1.22,32.14,33.76,80.0,0.04,1.44,6.49,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,50.0,H-LC,0.80,157610.0,-28485.0,75968.0,0.07,-15.31,48.20,25.51,15.0,-0.37,1.11,16.15,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,64.0,X-LC,1.95,203815.0,3677.0,17222.0,0.20,1.84,8.45,10.44,4.0,0.21,1.44,7.50,X40,NTT,FMCG
85,VOLTAS,1530.00,3.12,60.0,X-MC,2.80,216210.0,24468.0,13297.0,0.71,12.76,6.15,19.69,99.0,1.84,1.53,20.43,XY25,NTT,AC
29,HAVELLS,2069.16,-5.90,46.0,X-MC,2.91,233204.0,-14527.0,89574.0,-0.10,-5.86,38.41,30.30,92.0,-0.16,1.65,6.65,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.53,66.0,X-LC,3.18,226450.0,16084.0,12704.0,0.22,7.65,5.61,13.68,37.0,1.27,1.60,25.43,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51268.0,-13738.0,164668.0,-0.38,-21.13,321.19,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
3,ACC,3906.00,-37.64,50.0,X-MC,2.90,185010.0,-52941.0,205583.0,-0.30,-22.25,111.12,64.15,174.0,-0.26,1.31,3.78,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147324.0,-129199.0,239269.0,1.00,-46.72,162.41,39.81,54.0,-0.54,1.04,3.84,XY24,NTT,AUTO
21,COLPAL,3767.14,-4.10,46.0,X-MC,8.14,223940.0,-39425.0,152772.0,-2.08,-14.97,68.22,43.04,84.0,-0.26,1.58,3.96,XY25,ATH,FMCG
51,PAGEIND,51769.93,-27.42,40.0,X-MC,6.79,82050.0,-1460.0,21489.0,-1.05,-1.75,26.19,23.98,82.0,-0.07,0.58,3.97,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.76,51268.0,-13738.0,164668.0,-0.38,-21.13,321.19,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.42,40.0,X-MC,6.79,82050.0,-1460.0,21489.0,-1.05,-1.75,26.19,23.98,82.0,-0.07,0.58,3.97,X40,ATH,APPARELS
57,RELAXO,1176.00,-38.72,48.0,X-SC,4.65,83510.0,-61650.0,137583.0,0.10,-42.47,164.75,52.31,136.0,-0.45,0.59,11.78,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.04,44.0,X-MC,3.32,89790.0,-888.0,15794.0,-0.61,-0.98,17.59,16.44,101.0,-0.06,0.63,19.00,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.29,47.0,X-SC,4.44,96703.0,-31967.0,77266.0,-0.21,-24.84,79.90,35.20,219.0,-0.41,0.68,10.99,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.72,57.0,X-LC,12.36,287941.0,-58015.0,124707.0,-0.33,-16.77,43.31,19.28,1.0,-0.47,2.04,6.45,X40,ATH,IT
40,INFY,2275.00,-16.73,59.0,X-LC,7.17,324910.0,12022.0,159661.0,-0.20,3.84,49.14,54.87,3.0,0.08,2.30,10.75,X40,BTT,IT
42,ITC,452.00,-37.08,64.0,X-LC,1.95,203815.0,3677.0,17222.0,0.20,1.84,8.45,10.44,4.0,0.21,1.44,7.50,X40,NTT,FMCG
84,VBL,671.64,-18.24,53.0,X-LC,9.87,294373.0,-21469.0,134146.0,-0.89,-6.80,45.57,35.67,5.0,-0.16,2.08,6.12,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,47.0,H-LC,6.32,248688.0,-12931.0,132153.0,-0.01,-4.94,53.14,45.57,7.0,-0.10,1.76,5.82,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7050.00,62.0,L-SC,7.18,78459.0,-15351.0,91970.0,-2.70,-16.36,117.22,81.67,269.0,-0.17,0.55,51.94,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,27.02,82925.0,-30624.0,70677.0,2.64,-26.97,85.23,35.27,268.0,-0.43,0.59,102.96,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,52.0,H-SC,9.58,92460.0,-5520.0,27119.0,0.36,-5.63,29.33,22.05,152.0,-0.20,0.65,34.40,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,3.86,102795.0,11296.0,67454.0,-2.06,12.35,65.62,86.07,223.0,0.17,0.73,49.28,XR,NTT,DURABLES
46,KPIGREEN,731.05,9.40,71.0,H-SC,5.24,130460.0,5163.0,53763.0,1.79,4.12,41.21,47.03,141.0,0.10,0.92,63.35,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-197.73,81.0,M-SC,24.62,169425.0,20205.0,123968.0,7.94,13.54,73.17,96.62,208.0,0.16,1.20,83.35,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,122.87,69.0,H-SC,15.26,140454.0,-2025.0,21546.0,0.23,-1.42,15.34,13.70,163.0,-0.09,0.99,65.77,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.73,70.0,M-SC,9.60,83772.0,-17387.0,57384.0,0.14,-17.19,68.50,39.54,202.0,-0.30,0.59,18.13,AR,ATH,AUTO
5,ANGELONE,3033.00,16.33,71.0,X-SC,6.37,206189.0,15183.0,42516.0,0.14,7.95,20.62,30.21,157.0,0.36,1.46,30.49,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-55.29,72.0,H-SC,2.98,258324.0,-28181.0,285526.0,1.38,-9.84,110.53,89.82,134.0,-0.10,1.83,36.67,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.43
1,25,43.81
2,50,74.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    32.68
MC    22.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.29
X40      16.01
XY25     12.29
XR       11.32
X40N      9.32
AR        7.91
OX40N     7.62
X200      1.76
X5K       1.50
SR        1.09
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.68
X-LC    21.87
X-MC    17.08
M-SC    12.74
M-LC     5.15
X-SC     4.64
H-LC     4.63
H-MC     3.85
M-MC     1.58
L-SC     1.48
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.67,-3.80,37.67
IT,12.72,-16.86,79.86
FINANCE,12.25,-9.23,58.74
MISC,6.93,-17.35,82.37
BANKS,6.61,-6.71,62.84
PAINTS,5.64,-15.74,32.99
ELECTRICAL,5.30,-9.85,48.39
INSURANCE,3.88,-0.72,38.27
AUTO,2.81,-44.57,104.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3291125.0,22
XR,1307407.0,14
AR,1176192.0,9
X40,780068.0,12
X40N,611032.0,8
OX40N,569155.0,10
XY25,444352.0,8
SR,267798.0,2
X5K,105921.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3369918.0,25
M-SC,1205133.0,15
X-MC,1118506.0,13
X-LC,1045100.0,13
X-SC,524354.0,6
H-MC,335673.0,3
H-LC,282643.0,3
M-LC,280169.0,4
L-SC,252138.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1166959.0      6
           AR         844016.0      5
           XR         773769.0      7
M-SC       XY24       694384.0      6
X-MC       XY24       581657.0      4
X-LC       X40        448945.0      5
           XY24       308969.0      2
X-SC       X40N       282420.0      4
H-SC       SR         267798.0      2
           OX40N      263613.0      4
X-MC       X40        253857.0      6
X-LC       X40N       211689.0      3
H-LC       AR         208121.0      2
H-MC       XY24       180664.0      1
M-MC       XY24       169694.0      1
X-MC       XY25       166069.0      2
X-SC       XY24       164668.0      1
L-SC       XR         162647.0      2
M-SC       OX40N      147330.0      4
           AR         124055.0      2
           XY25       123968.0      1
X-MC       X40N       116923.0      1
M-SC       XR         115396.0      2
M-LC       XR         109747.0      1
           X5K        105921.0      1
L-SC       OX40N       89491.0      1
H-MC       XR          86288.0      1
X-SC       X40         77266.0      1
X-LC       XY25        75497.0      3
H-LC       X200        74522.0      1
H-MC       OX40N       68721.0      1
L-MC       XR          59560.0      1
H-SC       MH          53763.0      1
M-LC       XY25        40371.0      1
L-LC       XY25        38447.0      1
M-LC       XY24        24130.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
